<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [83]:
# avoid decoding problems
os.chdir("C:\\Users\\suman\\Downloads\\appliedaidataset\\Quora")
df = pd.read_csv("train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [84]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))


- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [86]:
#This step takes huge time
from scipy.sparse import csr_matrix,coo_matrix
from sklearn.cross_validation import train_test_split
df_all=df
#take sample
#_ , df = train_test_split(df, test_size = 5000, random_state=0,stratify = df['is_duplicate'] )
# Try TFIDF on texts
from scipy.sparse import hstack
tf_idf_vect = TfidfVectorizer(ngram_range=(1,3),min_df = 5,max_features = 50000)
final_tf_idf2 = tf_idf_vect.fit_transform(df['question2'])
tf_idf_vect = TfidfVectorizer(ngram_range=(1,3),min_df = 5,max_features = 50000)
final_tf_idf1 = tf_idf_vect.fit_transform(df['question1'])
print(final_tf_idf1.shape,final_tf_idf2.shape,df.shape)
df=df.drop(['id','question1','question2','qid1','qid2','is_duplicate'],axis=1)
print("last",df.head())
dense_matrix = np.array(df.as_matrix(columns = None), dtype=float).astype(np.float)
sparse_matrix = csr_matrix(dense_matrix)
dfnew=hstack((sparse_matrix, final_tf_idf1,final_tf_idf2)).tocsr()
print(dfnew.shape)

(404290, 50000) (404290, 50000) (404290, 6)
last Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]
(404290, 100000)


In [87]:
type(dfnew)

scipy.sparse.csr.csr_matrix

In [88]:
df=dfnew
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [89]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
#df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df
#df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
#df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)
#df3_q1 = df3.q1_feats_m
#df3_q2 = df3.q2_feats_m

In [90]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [91]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [92]:
# Questions 1 tfidf weighted word2vec
#df3_q1.head()
print(df3.shape)
#pd.DataFrame(df3.toarray()).head()

(404290, 100000)


In [93]:
# Questions 2 tfidf weighted word2vec
#df3_q2.head()

In [96]:
print("Number of features in nlp dataframe :", df1.shape)
print("Number of features in preprocessed dataframe :", df2.shape)
#print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
#print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in tfidf for Q1 and Q2 :", df3.shape)
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3.shape[1])

Number of features in nlp dataframe : (404290, 28)
Number of features in preprocessed dataframe : (404290, 12)
Number of features in tfidf for Q1 and Q2 : (404290, 100000)
Number of features in final dataframe  : 100040


In [95]:
# storing the final features to csv file
from scipy import sparse
import os
os.chdir("C:\\Users\\suman\\Downloads\\appliedaidataset\\Quora")

#if not os.path.isfile('final_features.csv'):
if not os.path.isfile('dfnew.npz'):
    #df3_q1['id']=df1['id']
    #df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    #df2  = df3_q1.merge(df3_q2, on='id',how='left')
    print(df1.head())
    df1=df1.drop(['id'],axis=1)
    #result  = df1.merge(df2, on='id',how='left')
    #instead of tdidf use normal tfidf
    #result  = df1.merge(df2, on='id',how='left')
    print(df1.shape,df1.shape)
    dense_matrix = np.array(df1.as_matrix(columns = None), dtype=float).astype(np.float)
    sparse_matrix = csr_matrix(dense_matrix)
    print("last",df1.head())
    dfnew=hstack((sparse_matrix, df3)).tocsr()
    print("final shape",dfnew.shape)

    #dfnew.to_csv('final_features.csv')
    sparse.save_npz("dfnew.npz", dfnew)
else:
    your_matrix_back = sparse.load_npz("dfnew.npz")
    print(your_matrix_back.shape)


   id  is_duplicate   cwc_min   cwc_max   csc_min   csc_max   ctc_min  \
0   0             0  0.999980  0.833319  0.999983  0.999983  0.916659   
1   1             0  0.799984  0.399996  0.749981  0.599988  0.699993   
2   2             0  0.399992  0.333328  0.399992  0.249997  0.399996   
3   3             0  0.000000  0.000000  0.000000  0.000000  0.000000   
4   4             0  0.399992  0.199998  0.999950  0.666644  0.571420   

    ctc_max  last_word_eq  first_word_eq     ...      freq_qid2  q1len  q2len  \
0  0.785709           0.0            1.0     ...              1     66     57   
1  0.466664           0.0            1.0     ...              1     51     88   
2  0.285712           0.0            1.0     ...              1     73     59   
3  0.000000           0.0            0.0     ...              1     50     65   
4  0.307690           0.0            1.0     ...              1     76     39   

   q1_n_words  q2_n_words  word_Common  word_Total  word_share  freq_q1+q2

KeyError: "labels ['id'] not contained in axis"